In [1]:
%load_ext autoreload
%autoreload 2

In [5]:
# Code to set up the assignment
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/

!pip3 install pybind11

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
!make

In [ ]:
!python3 -m pytest -l -v -k "pad_forward"

In [2]:
import sys
sys.path.append('./python')

In [9]:
import needle as ndl
import needle.ops as ops
import numpy as np
import torch

In [16]:
T, d_model = 100, 64
val = np.random.random((1, T, d_model))

In [17]:
mask = torch.triu(-float("inf")*torch.ones(T,T),1).numpy()

In [50]:
a = np.random.random((4, 3))
a

array([[0.19604483, 0.21950351, 0.31655932],
       [0.35103041, 0.79442739, 0.14689571],
       [0.62632848, 0.10272157, 0.26743422],
       [0.82095482, 0.70563079, 0.49788598]])

In [101]:
np.max(a, -1).shape, np.max(a, -1, keepdims=True).shape

((4,), (4, 1))

In [57]:
ndl_tensor  = ndl.Tensor(a)

In [59]:
ndl_tensor.maximum(), ndl_tensor.maximum(0), ndl_tensor.maximum(-1)

(needle.Tensor(0.8209548),
 needle.Tensor([0.8209548  0.7944274  0.49788597]),
 needle.Tensor([0.3165593  0.7944274  0.62632847 0.8209548 ]))

In [95]:
ndl_tensor = ndl.Tensor(a, requires_grad=True)
b = ndl_tensor.maximum(0)
out = b.sum()
out.backward()
ndl_tensor.grad

out_grad: [1. 1. 1.], out_grad_shape: (3,) out_shape: (1, 3),  in_shape: (4, 3)


needle.Tensor([1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.])

In [96]:
b

needle.Tensor([0.8209548  0.7944274  0.49788597])

In [88]:
torch_tensor = torch.tensor(a, requires_grad=True)
torch_b = torch.softmax(torch_tensor, 0)
out = torch_b.sum()
out.backward()
torch_tensor.grad

tensor([[0.0000e+00, 2.0976e-17, 0.0000e+00],
        [0.0000e+00, 3.7275e-17, 0.0000e+00],
        [0.0000e+00, 1.8664e-17, 0.0000e+00],
        [0.0000e+00, 3.4107e-17, 0.0000e+00]], dtype=torch.float64)

In [93]:
torch_b

tensor([[0.1794, 0.1889, 0.2503],
        [0.2095, 0.3357, 0.2113],
        [0.2759, 0.1681, 0.2383],
        [0.3352, 0.3072, 0.3001]], dtype=torch.float64,
       grad_fn=<SoftmaxBackward0>)

In [117]:
T, d = 100, 128
attn = torch.nn.MultiheadAttention(d, 1, bias=False, batch_first=True)
N = 10
M = torch.triu(-float("inf")*torch.ones(T,T),1)
X = torch.randn(N, T, d)
Y_, A_ = attn(X,X,X, attn_mask=M)

In [118]:
W_KQV = attn.in_proj_weight.detach().numpy().T

In [119]:
X.shape

torch.Size([10, 100, 128])

In [120]:
W_KQV.shape

(128, 384)

In [122]:
W_out = attn.out_proj.weight.detach().numpy().T
W_out.shape

(128, 128)

In [ ]:
X

In [126]:
T, d = 100, 128
heads = 4
N = 10
M = torch.triu(-float("inf") * torch.ones(T, T), 1).numpy()
X = np.random.random((N, T, d))

In [132]:
torch_x = torch.tensor(X, dtype=torch.float32)
torch_mask = torch.tensor(M, dtype=torch.float32)

attn = torch.nn.MultiheadAttention(d, heads, bias=False, batch_first=True)
Y_, A_ = attn(torch_x, torch_x, torch_x, attn_mask=torch_mask)

In [142]:
w_kqv = ndl.nn.Parameter(attn.in_proj_weight.detach().numpy().T)
w_out = ndl.nn.Parameter(attn.out_proj.weight.detach().numpy().T)

In [156]:
ndl_x.shape

(10, 100, 128)

In [157]:
ndl_multihead_attention = ndl.nn.MultiHeadedAttention(heads, d)
ndl_multihead_attention.weight = w_kqv
ndl_multihead_attention.w_out = w_out

ndl_x = ndl.Tensor(X)
ndl_mask = ndl.Tensor(M)

Y, A = ndl_multihead_attention(ndl_x, ndl_mask)

x @ self.weight (10, 100, 384)


TypeError: split() got multiple values for argument 'axis'

In [160]:
ndl.Tensor(np.arange(9))

needle.Tensor([0. 1. 2. 3. 4. 5. 6. 7. 8.])

In [163]:
ndl.ops.split(ndl.Tensor(np.arange(9)), axis=-1)

TypeError: _split_dispatcher() missing 1 required positional argument: 'indices_or_sections'